1.3: Pipelines
================
### Stringing it All Together
_____________________

**Pipelines** has a few different meanings in Data Science.

This creates some ambiguity as the terms are often used interchangably.

There's [Spark Pipelines](https://spark.apache.org/docs/2.2.0/ml-pipeline.html), used to process data in PySpark. 

Then there's the overarching idea of the Data Science Pipeline: a concept in business that describes the general process for going from raw data to something that creates value for the business, as this [example](https://www.ibm.com/developerworks/library/ba-intro-data-science-1/index.html) from IBM illustrates.

Then, there is the scikit-learn Pipeline modeule (term?). This can be described as a simple way, using scikit-learn, to work through the beginning/middle steps of the Data Science Pipeline. **In scikit-learn, a Pipeline is a string of transforms with a final estimator.** All of these steps in the pipeline (transformers and estimators) can be user-defined or from scikit-learn.

**You can make user-defined pipeline steps by inheriting from the TransformerMixin and BaseEstimator parent classes**. If you inherit from these and write your transformer or estimator properly, they will work together in the pipeline seamlessly!

